# Analysis of TextualVerifier Only

In [1]:
# Setup and Imports
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as scp
# import plotly.express as px
import matplotlib.pyplot as plt
from numpy.polynomial.polynomial import polyfit

## Load Results

In [2]:
df_textualverifier_1v = pd.read_csv("results/textualverifier-1v.csv")
df_textualverifier_2v = pd.read_csv("results/textualverifier-2v.csv")
df_textualverifier_3v = pd.read_csv("results/textualverifier-3v.csv")
df_textualverifier_4v = pd.read_csv("results/textualverifier-4v.csv")
df_textualverifier_5v = pd.read_csv("results/textualverifier-5v.csv")